## Step 1: Install Dependencies

In [1]:
!pip install langchain langchain-community langchain-openai langchain-text-splitters -q
!pip install chromadb sentence-transformers rank-bm25 -q
!pip install google-generativeai -q
print("All packages installed!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Step 2: Load TechCorp Dataset

In [2]:
TECHCORP_DOCS = {
    "employee_handbook": """
# TechCorp Employee Handbook

## Chapter 2: Employment Policies

### 2.1 Working Hours
Standard Hours: Monday to Friday, 9:00 AM to 6:00 PM local time.
Flexible Work Policy: Core hours are 10:00 AM to 4:00 PM. Outside core hours, employees may adjust their schedule with manager approval.
Remote Work: Employees may work remotely up to 3 days per week. Fully remote positions require VP approval.

### 2.2 Time Off & Leave

#### Paid Time Off (PTO)
| Tenure | Annual PTO |
| 0-2 years | 15 days |
| 2-5 years | 20 days |
| 5+ years | 25 days |

PTO does not roll over. Maximum carryover: 5 days with manager approval.

#### Sick Leave
Employees receive 10 days of paid sick leave annually. Sick leave does not roll over.

#### Parental Leave
Primary caregiver: 16 weeks paid leave
Secondary caregiver: 6 weeks paid leave
Adoption and surrogacy receive equal benefits.

### 2.3 Performance Reviews
Annual reviews conducted in Q4. Mid-year check-ins in June.
Rating scale: 1 (Needs Improvement) to 5 (Exceptional)
Promotion eligibility requires minimum rating of 3.5 and 18 months in current role.
""",

    "technical_architecture": """
# TechCorp Technical Architecture Document

## 1. System Overview

TechCorp's platform processes 2.3 million API requests daily across 12 global regions.
Primary tech stack: Python (FastAPI), PostgreSQL, Redis, Kubernetes on AWS.

## 2. Service Architecture

### 2.1 Core Services
- **AuthService**: Handles OAuth 2.0 + OIDC authentication. 99.99% uptime SLA.
- **DataPipeline**: Apache Kafka for event streaming. Processes 50K events/second.
- **MLInference**: Model serving via TensorFlow Serving. Average latency: 45ms.
- **SearchService**: Elasticsearch cluster. 500M documents indexed.

### 2.2 Database Architecture
Primary: PostgreSQL 15 with read replicas in each region.
Cache: Redis Cluster (6 nodes) with 99.95% hit rate.
Data Warehouse: Snowflake for analytics. 50TB+ data.

## 3. Deployment & CI/CD

### 3.1 Deployment Strategy
Blue-green deployments for zero-downtime releases.
Canary releases: 5% → 25% → 100% rollout over 4 hours.
Rollback time: Under 2 minutes.

### 3.2 CI/CD Pipeline
GitHub Actions → Docker Build → ECR → ArgoCD → EKS
Average deployment time: 8 minutes from merge to production.

## 4. Monitoring & Observability

### 4.1 Monitoring Stack
Prometheus + Grafana for metrics
Jaeger for distributed tracing
PagerDuty for alerting (P1 response: 5 minutes)

### 4.2 SLAs
API Gateway: 99.95% availability
Core Services: 99.9% availability
Data Pipeline: 99.5% availability (eventual consistency)
""",

    "security_policies": """
# TechCorp Security Policies

## 1. Access Control

### 1.1 Authentication Requirements
- All employees must use SSO via Okta
- MFA required for all accounts (hardware key preferred)
- Session timeout: 8 hours for internal apps, 1 hour for admin consoles
- Password requirements: 16+ characters, no reuse of last 12 passwords

### 1.2 Authorization Model
Role-Based Access Control (RBAC) with principle of least privilege.
Access reviews conducted quarterly by Security team.
Privileged access requires manager + Security approval.

## 2. Data Classification

### 2.1 Classification Levels
- **Public**: Marketing materials, blog posts
- **Internal**: Employee handbook, internal wikis
- **Confidential**: Customer data, financial reports
- **Restricted**: Encryption keys, auth tokens, PII

### 2.2 Handling Requirements
Confidential data: Encryption at rest (AES-256) and in transit (TLS 1.3)
Restricted data: Additional access logging, no local storage permitted

## 3. Incident Response

### 3.1 Severity Levels
- **P1 (Critical)**: Data breach, system-wide outage. Response: 15 minutes.
- **P2 (High)**: Service degradation, security vulnerability. Response: 1 hour.
- **P3 (Medium)**: Minor issues, non-critical bugs. Response: 4 hours.
- **P4 (Low)**: Cosmetic issues, minor improvements. Response: 24 hours.

### 3.2 Reporting
Security incidents: security@techcorp.com or #security-incidents Slack
All employees must report suspected incidents within 1 hour.

## 4. Compliance

SOC 2 Type II certified
GDPR compliant for EU customers
HIPAA compliant for healthcare data
Annual penetration testing by third party (last: October 2024)
""",

    "org_structure": """
# TechCorp Organization Structure

## Executive Team
- **CEO**: Sarah Chen - Founded TechCorp in 2018
- **CTO**: Marcus Rodriguez - Former Google, leads 200+ engineers
- **CFO**: Jennifer Walsh - IPO preparation lead
- **CPO**: David Kim - Product strategy and roadmap
- **CISO**: Aisha Patel - Security and compliance

## Engineering Organization

### Platform Engineering (Director: Kevin Patel)
- Infrastructure Team (12 engineers)
- DevOps Team (8 engineers)
- SRE Team (6 engineers)

### Product Engineering (Director: Lisa Wang)
- Frontend Team (15 engineers)
- Backend Team (20 engineers)
- Mobile Team (10 engineers)

### AI/ML Team (Director: Dr. James Liu)
- ML Platform (8 engineers)
- Applied ML (12 engineers)
- Research (5 engineers)

## Key Contacts
- IT Support: it-help@techcorp.com or ext. 4357
- HR Questions: hr@techcorp.com
- Security Issues: security@techcorp.com
- Facilities: facilities@techcorp.com

## Office Locations
- HQ: San Francisco, CA (500 employees)
- Engineering Hub: Austin, TX (150 employees)
- Sales Office: New York, NY (75 employees)
- R&D Center: London, UK (50 employees)
"""
}

print("  TechCorp dataset loaded!")
print(f" Documents: {len(TECHCORP_DOCS)}")
print(f" Total chars: {sum(len(v) for v in TECHCORP_DOCS.values()):,}")


  TechCorp dataset loaded!
 Documents: 4
 Total chars: 5,283


## Step 3: Prepare Chunks and Base Retrieval

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Chunk the documents
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n## ", "\n### ", "\n\n", "\n", " "]
)

all_chunks = []
chunk_metadata = []

for doc_name, content in TECHCORP_DOCS.items():
    chunks = splitter.split_text(content)
    for i, chunk in enumerate(chunks):
        all_chunks.append(chunk)
        chunk_metadata.append({
            "source": doc_name,
            "chunk_id": i,
            "char_count": len(chunk)
        })

print(f"Created {len(all_chunks)} chunks")

# Initialize models
print("Loading models...")
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# Create embeddings
print("Creating embeddings...")
embeddings = embed_model.encode(all_chunks, show_progress_bar=True)

# Create BM25 index
tokenized_chunks = [chunk.lower().split() for chunk in all_chunks]
bm25 = BM25Okapi(tokenized_chunks)

print("Base retrieval ready!")

Created 19 chunks
Loading models...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Creating embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Base retrieval ready!


## Step 4: Baseline - Hybrid Search (Day 13 Recap)

In [4]:
def hybrid_search(query, top_k=5, bm25_weight=0.35):
    """
    Day 13's hybrid search as our baseline.
    Combines BM25 + vector search + re-ranking.
    """
    # BM25 scores
    bm25_scores = bm25.get_scores(query.lower().split())
    bm25_norm = bm25_scores / (max(bm25_scores) + 1e-6)

    # Vector scores
    query_emb = embed_model.encode([query])
    vector_scores = cosine_similarity(query_emb, embeddings)[0]

    # Combine
    combined = bm25_weight * bm25_norm + (1 - bm25_weight) * vector_scores

    # Get top candidates for re-ranking
    top_idx = np.argsort(combined)[::-1][:top_k * 3]
    candidates = [(all_chunks[i], chunk_metadata[i], combined[i]) for i in top_idx]

    # Re-rank
    pairs = [[query, c[0]] for c in candidates]
    rerank_scores = reranker.predict(pairs)

    results = []
    for i, (chunk, meta, orig_score) in enumerate(candidates):
        results.append({
            "chunk": chunk,
            "metadata": meta,
            "hybrid_score": float(orig_score),
            "rerank_score": float(rerank_scores[i])
        })

    results = sorted(results, key=lambda x: x["rerank_score"], reverse=True)
    return results[:top_k]

# Test baseline
print("Testing baseline hybrid search...")
test_results = hybrid_search("What is the PTO policy?")
print(f"Baseline working! Top result from: {test_results[0]['metadata']['source']}")


Testing baseline hybrid search...
Baseline working! Top result from: employee_handbook


## Step 5: HyDE - Hypothetical Document Embeddings

In [5]:
"""
HyDE (Hypothetical Document Embeddings)
Paper: "Precise Zero-Shot Dense Retrieval without Relevance Labels" (2022)

The idea:
1. User asks: "What's the parental leave policy?"
2. LLM generates a HYPOTHETICAL answer (even if wrong)
3. Embed that hypothetical answer
4. Search using the hypothetical embedding (not the question!)

Why it works:
- Questions and answers live in different embedding spaces
- A hypothetical answer is closer to real answers in embedding space
- Even incorrect details don't matter - the semantic region is right
"""

import google.generativeai as genai
from google.colab import userdata

# Configure Gemini (free tier)
try:
    genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
    llm = genai.GenerativeModel('gemini-1.5-flash')
    LLM_AVAILABLE = True
    print("Gemini API configured!")
except:
    LLM_AVAILABLE = False
    print("   No API key found. Using mock LLM responses for demo.")
    print("   To enable: Add GOOGLE_API_KEY to Colab secrets")

def generate_hypothetical_answer(query):
    """Generate a hypothetical document that would answer the query."""

    if not LLM_AVAILABLE:
        # Mock response for demo
        return f"Based on company policy, the answer to '{query}' would typically include specific details about procedures, timelines, and requirements as documented in the employee handbook or technical documentation."

    prompt = f"""You are a helpful assistant. Generate a hypothetical paragraph that would
appear in a company document answering this question. Don't say you don't know -
just write what such a document WOULD contain.

Question: {query}

Hypothetical document paragraph:"""

    response = llm.generate_content(prompt)
    return response.text

def hyde_search(query, top_k=5):
    """
    HyDE: Search using hypothetical document embedding.

    Returns both standard and HyDE results for comparison.
    """
    # Generate hypothetical answer
    hypothetical = generate_hypothetical_answer(query)

    # Embed the hypothetical (not the query!)
    hyde_embedding = embed_model.encode([hypothetical])

    # Also get standard query embedding for comparison
    query_embedding = embed_model.encode([query])

    # Search with HyDE embedding
    hyde_scores = cosine_similarity(hyde_embedding, embeddings)[0]
    standard_scores = cosine_similarity(query_embedding, embeddings)[0]

    # Get top results
    hyde_top_idx = np.argsort(hyde_scores)[::-1][:top_k]
    standard_top_idx = np.argsort(standard_scores)[::-1][:top_k]

    hyde_results = [{
        "chunk": all_chunks[i],
        "metadata": chunk_metadata[i],
        "score": float(hyde_scores[i])
    } for i in hyde_top_idx]

    standard_results = [{
        "chunk": all_chunks[i],
        "metadata": chunk_metadata[i],
        "score": float(standard_scores[i])
    } for i in standard_top_idx]

    return {
        "hypothetical": hypothetical,
        "hyde_results": hyde_results,
        "standard_results": standard_results
    }

# Test HyDE
print("\n" + "="*60)
print(" Testing HyDE Search")
print("="*60)

test_query = "How long is parental leave for new fathers?"
hyde_output = hyde_search(test_query)

print(f"\nQuery: {test_query}")
print(f"\n Hypothetical Answer (generated):")
print(f"   {hyde_output['hypothetical'][:200]}...")
print(f"\n HyDE Top Result:")
print(f"   Source: {hyde_output['hyde_results'][0]['metadata']['source']}")
print(f"   Score: {hyde_output['hyde_results'][0]['score']:.4f}")
print(f"\n Standard Top Result:")
print(f"   Source: {hyde_output['standard_results'][0]['metadata']['source']}")
print(f"   Score: {hyde_output['standard_results'][0]['score']:.4f}")


   No API key found. Using mock LLM responses for demo.
   To enable: Add GOOGLE_API_KEY to Colab secrets

 Testing HyDE Search

Query: How long is parental leave for new fathers?

 Hypothetical Answer (generated):
   Based on company policy, the answer to 'How long is parental leave for new fathers?' would typically include specific details about procedures, timelines, and requirements as documented in the employe...

 HyDE Top Result:
   Source: employee_handbook
   Score: 0.4572

 Standard Top Result:
   Source: employee_handbook
   Score: 0.5246


## Step 6: Self-RAG - Self-Reflective Retrieval

In [6]:
"""
Self-RAG (Self-Reflective RAG)
Paper: "Self-RAG: Learning to Retrieve, Generate, and Critique" (2023)

The idea:
1. Retrieve documents
2. LLM evaluates: "Is this retrieval relevant?"
3. Generate answer with citations
4. LLM evaluates: "Is this answer supported by the documents?"
5. If not supported → retrieve more or regenerate

Key tokens the model learns:
- [Retrieve]: Should I retrieve?
- [IsRel]: Is this document relevant?
- [IsSup]: Is this claim supported?
- [IsUse]: Is this response useful?
"""

def evaluate_relevance(query, chunk):
    """Have LLM evaluate if a chunk is relevant to the query."""

    if not LLM_AVAILABLE:
        # Simple heuristic for demo
        query_words = set(query.lower().split())
        chunk_words = set(chunk.lower().split())
        overlap = len(query_words & chunk_words)
        return overlap >= 2, f"Word overlap: {overlap}"

    prompt = f"""Evaluate if this document chunk is relevant to answering the question.

Question: {query}

Document chunk:
{chunk[:500]}

Is this chunk relevant? Respond with only "YES" or "NO" followed by a brief reason.
"""

    response = llm.generate_content(prompt)
    text = response.text.strip().upper()
    is_relevant = text.startswith("YES")
    return is_relevant, response.text

def evaluate_support(query, answer, chunks):
    """Evaluate if the answer is supported by the retrieved chunks."""

    if not LLM_AVAILABLE:
        return True, "Mock evaluation: Answer appears supported"

    context = "\n---\n".join(chunks[:3])

    prompt = f"""Evaluate if this answer is fully supported by the provided documents.

Question: {query}

Answer: {answer}

Documents:
{context[:1500]}

Is the answer supported? Respond with "SUPPORTED", "PARTIALLY_SUPPORTED", or "NOT_SUPPORTED"
followed by explanation.
"""

    response = llm.generate_content(prompt)
    text = response.text.strip().upper()
    is_supported = "SUPPORTED" in text and "NOT_SUPPORTED" not in text
    return is_supported, response.text

def self_rag_search(query, top_k=5, max_iterations=2):
    """
    Self-RAG: Retrieve, evaluate, and iterate.

    Process:
    1. Retrieve initial candidates
    2. Filter by relevance (LLM evaluation)
    3. If not enough relevant → retrieve more
    4. Generate answer
    5. Check if supported → if not, iterate
    """

    results = {
        "iterations": [],
        "final_chunks": [],
        "relevance_scores": []
    }

    # Initial retrieval
    initial_results = hybrid_search(query, top_k=top_k * 2)

    for iteration in range(max_iterations):
        iter_data = {"iteration": iteration + 1, "evaluated": []}

        relevant_chunks = []
        for r in initial_results:
            is_relevant, reason = evaluate_relevance(query, r["chunk"])
            iter_data["evaluated"].append({
                "chunk_preview": r["chunk"][:100] + "...",
                "is_relevant": is_relevant,
                "reason": reason[:100] if isinstance(reason, str) else str(reason)[:100]
            })

            if is_relevant:
                relevant_chunks.append(r)

        iter_data["relevant_count"] = len(relevant_chunks)
        results["iterations"].append(iter_data)

        # If we have enough relevant chunks, stop
        if len(relevant_chunks) >= top_k:
            results["final_chunks"] = relevant_chunks[:top_k]
            break

        # Otherwise, we'd retrieve more (simplified: just use what we have)
        results["final_chunks"] = relevant_chunks if relevant_chunks else initial_results[:top_k]
        break

    return results

# Test Self-RAG
print("\n" + "="*60)
print(" Testing Self-RAG Search")
print("="*60)

test_query = "What is the incident response time for critical issues?"
selfrag_output = self_rag_search(test_query)

print(f"\nQuery: {test_query}")
print(f"\n Self-RAG Results:")
print(f"   Iterations: {len(selfrag_output['iterations'])}")
print(f"   Chunks evaluated: {len(selfrag_output['iterations'][0]['evaluated'])}")
print(f"   Relevant found: {selfrag_output['iterations'][0]['relevant_count']}")
print(f"\n Final chunks: {len(selfrag_output['final_chunks'])}")



 Testing Self-RAG Search

Query: What is the incident response time for critical issues?

 Self-RAG Results:
   Iterations: 1
   Chunks evaluated: 10
   Relevant found: 1

 Final chunks: 1


## Step 7: Corrective RAG (CRAG)

In [7]:
"""
Corrective RAG (CRAG)
Paper: "Corrective Retrieval Augmented Generation" (2024)

The idea:
1. Retrieve documents
2. Evaluate retrieval quality (Correct / Ambiguous / Incorrect)
3. If Correct → use retrieved docs
4. If Ambiguous → refine query and retrieve again
5. If Incorrect → fall back to web search

This handles the "knowledge gap" problem gracefully.
"""

def evaluate_retrieval_quality(query, chunks):
    """
    Evaluate overall retrieval quality.
    Returns: 'correct', 'ambiguous', or 'incorrect'
    """

    if not LLM_AVAILABLE:
        # Simple heuristic for demo
        if not chunks:
            return 'incorrect', "No chunks retrieved"

        # Check if query terms appear in chunks
        query_terms = set(query.lower().split())
        chunk_text = " ".join([c["chunk"].lower() for c in chunks[:3]])
        matches = sum(1 for term in query_terms if term in chunk_text)

        if matches >= len(query_terms) * 0.6:
            return 'correct', f"Good term overlap ({matches}/{len(query_terms)})"
        elif matches >= len(query_terms) * 0.3:
            return 'ambiguous', f"Partial term overlap ({matches}/{len(query_terms)})"
        else:
            return 'incorrect', f"Low term overlap ({matches}/{len(query_terms)})"

    context = "\n---\n".join([c["chunk"][:300] for c in chunks[:3]])

    prompt = f"""Evaluate if these retrieved documents can answer the question.

Question: {query}

Retrieved documents:
{context}

Classify as:
- CORRECT: Documents contain the answer
- AMBIGUOUS: Documents are partially relevant but incomplete
- INCORRECT: Documents are not relevant

Respond with only the classification and brief reason.
"""

    response = llm.generate_content(prompt)
    text = response.text.strip().upper()

    if "CORRECT" in text and "INCORRECT" not in text:
        return 'correct', response.text
    elif "AMBIGUOUS" in text:
        return 'ambiguous', response.text
    else:
        return 'incorrect', response.text

def refine_query(original_query):
    """Refine query for better retrieval."""

    if not LLM_AVAILABLE:
        # Simple expansion
        return f"{original_query} policy procedure details"

    prompt = f"""The following query didn't return good results.
Rewrite it to be more specific and likely to match relevant documents.

Original query: {original_query}

Refined query (just the query, nothing else):"""

    response = llm.generate_content(prompt)
    return response.text.strip()

def mock_web_search(query):
    """Mock web search fallback."""
    return [{
        "source": "web_search",
        "chunk": f"[Web Search Result] Based on general best practices, {query.lower()} typically involves following industry standards and organizational guidelines. Specific details would depend on your organization's policies.",
        "metadata": {"source": "web_fallback", "note": "Fallback when internal docs insufficient"}
    }]

def crag_search(query, top_k=5):
    """
    Corrective RAG: Evaluate and correct retrieval.

    Flow:
    1. Initial retrieval
    2. Evaluate quality
    3. Correct based on evaluation
    """

    result = {
        "original_query": query,
        "actions": [],
        "final_chunks": []
    }

    # Step 1: Initial retrieval
    initial_results = hybrid_search(query, top_k=top_k)
    result["actions"].append({"step": "initial_retrieval", "count": len(initial_results)})

    # Step 2: Evaluate quality
    quality, reason = evaluate_retrieval_quality(query, initial_results)
    result["actions"].append({
        "step": "quality_evaluation",
        "quality": quality,
        "reason": reason[:200] if isinstance(reason, str) else str(reason)[:200]
    })

    # Step 3: Take corrective action
    if quality == 'correct':
        result["final_chunks"] = initial_results
        result["actions"].append({"step": "use_retrieved", "action": "Documents are good, using as-is"})

    elif quality == 'ambiguous':
        # Refine query and try again
        refined = refine_query(query)
        result["actions"].append({"step": "refine_query", "refined": refined})

        refined_results = hybrid_search(refined, top_k=top_k)

        # Combine original and refined results
        seen_chunks = set()
        combined = []
        for r in initial_results + refined_results:
            chunk_hash = hash(r["chunk"][:100])
            if chunk_hash not in seen_chunks:
                seen_chunks.add(chunk_hash)
                combined.append(r)

        result["final_chunks"] = combined[:top_k]
        result["actions"].append({"step": "combined_results", "count": len(result["final_chunks"])})

    else:  # incorrect
        # Fall back to web search
        result["actions"].append({"step": "web_fallback", "reason": "Internal docs insufficient"})
        web_results = mock_web_search(query)

        # Combine any marginally relevant internal docs with web results
        result["final_chunks"] = web_results + initial_results[:2]
        result["actions"].append({"step": "fallback_complete", "sources": "web + partial internal"})

    return result

# Test CRAG with different queries
print("\n" + "="*60)
print(" Testing Corrective RAG (CRAG)")
print("="*60)

test_queries = [
    "What is the PTO policy for employees with 3 years tenure?",  # Should be correct
    "How do we handle GDPR data requests?",  # Might be ambiguous
    "What is the company's Mars colonization strategy?"  # Should be incorrect
]

for query in test_queries:
    print(f"\n{'='*50}")
    print(f"Query: {query}")
    crag_output = crag_search(query)

    for action in crag_output["actions"]:
        print(f"   → {action['step']}: {action.get('quality', action.get('action', action.get('count', '')))}")

    print(f"   Final chunks: {len(crag_output['final_chunks'])}")



 Testing Corrective RAG (CRAG)

Query: What is the PTO policy for employees with 3 years tenure?
   → initial_retrieval: 5
   → quality_evaluation: correct
   → use_retrieved: Documents are good, using as-is
   Final chunks: 5

Query: How do we handle GDPR data requests?
   → initial_retrieval: 5
   → quality_evaluation: ambiguous
   → refine_query: 
   → combined_results: 5
   Final chunks: 5

Query: What is the company's Mars colonization strategy?
   → initial_retrieval: 5
   → quality_evaluation: incorrect
   → web_fallback: 
   → fallback_complete: 
   Final chunks: 3


### Step 8: When to Use Each Pattern

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                    ADVANCED RETRIEVAL PATTERNS - DECISION GUIDE              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  🔷 HYBRID SEARCH (Day 13 - Our Baseline)                                    ║
║     ├─ When: Default choice for most RAG applications                        ║
║     ├─ Best for: Queries with specific terms + semantic meaning              ║
║     ├─ Cost: Low (no LLM calls for retrieval)                                ║
║     └─ Example: "What is Kevin Patel's role?"                                ║
║                                                                              ║
║  🔷 HyDE (Hypothetical Document Embeddings)                                  ║
║     ├─ When: Complex questions where query ≠ answer format                   ║
║     ├─ Best for: "How" and "Why" questions, explanatory queries              ║
║     ├─ Cost: Medium (1 LLM call per query)                                   ║
║     ├─ Trade-off: Better recall vs. added latency                            ║
║     └─ Example: "How should I prepare for my performance review?"            ║
║                                                                              ║
║  🔷 SELF-RAG (Self-Reflective RAG)                                           ║
║     ├─ When: High-stakes queries requiring verified answers                  ║
║     ├─ Best for: Compliance, legal, medical contexts                         ║
║     ├─ Cost: High (multiple LLM calls for evaluation)                        ║
║     ├─ Trade-off: Higher accuracy vs. 2-3x latency                           ║
║     └─ Example: "What are the HIPAA compliance requirements?"                ║
║                                                                              ║
║  🔷 CRAG (Corrective RAG)                                                    ║
║     ├─ When: Knowledge base might have gaps                                  ║
║     ├─ Best for: General Q&A where users might ask anything                  ║
║     ├─ Cost: Variable (depends on retrieval quality)                         ║
║     ├─ Trade-off: Graceful degradation vs. complexity                        ║
║     └─ Example: "What's the industry standard for API rate limiting?"        ║
║                                                                              ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                           PRODUCTION RECOMMENDATION                          ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║   Start with: Hybrid Search + Re-ranking (Day 13)                            ║
║   Add HyDE: If you see poor recall on complex questions                      ║
║   Add Self-RAG: For compliance/high-stakes domains                           ║
║   Add CRAG: If users frequently ask out-of-scope questions                   ║
║                                                                              ║
║   🎯 Most production systems: Hybrid + selective HyDE = 90% of value         ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")


## Step 9: Production-Ready Advanced RAG Class

In [ ]:
class AdvancedRAG:
    """
    Production-ready RAG with multiple retrieval strategies.

    Strategies:
    - standard: Basic vector search
    - hybrid: BM25 + vector + re-ranking (Day 13)
    - hyde: Hypothetical document embeddings
    - self_rag: Self-reflective with evaluation
    - crag: Corrective with fallback
    - auto: Automatically choose based on query
    """

    def __init__(self, chunks, metadata):
        self.chunks = chunks
        self.metadata = metadata

        # Initialize models
        self.embed_model = SentenceTransformer("all-MiniLM-L6-v2")
        self.reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

        # Create indices
        self.embeddings = self.embed_model.encode(chunks)
        tokenized = [c.lower().split() for c in chunks]
        self.bm25 = BM25Okapi(tokenized)

        # LLM for advanced patterns
        self.llm_available = LLM_AVAILABLE

    def search(self, query, strategy="auto", top_k=5):
        """
        Search with specified strategy.

        Args:
            query: Search query
            strategy: 'standard', 'hybrid', 'hyde', 'self_rag', 'crag', 'auto'
            top_k: Number of results

        Returns:
            dict with results and metadata
        """
        if strategy == "auto":
            strategy = self._choose_strategy(query)

        if strategy == "standard":
            return self._standard_search(query, top_k)
        elif strategy == "hybrid":
            return self._hybrid_search(query, top_k)
        elif strategy == "hyde":
            return self._hyde_search(query, top_k)
        elif strategy == "self_rag":
            return self._self_rag_search(query, top_k)
        elif strategy == "crag":
            return self._crag_search(query, top_k)
        else:
            return self._hybrid_search(query, top_k)

    def _choose_strategy(self, query):
        """Auto-select strategy based on query characteristics."""
        query_lower = query.lower()

        # Complex "how/why" questions → HyDE
        if query_lower.startswith(("how ", "why ", "explain ", "what is the process")):
            return "hyde"

        # Questions about compliance/policy → Self-RAG
        compliance_terms = ["compliance", "legal", "regulation", "hipaa", "gdpr", "soc"]
        if any(term in query_lower for term in compliance_terms):
            return "self_rag"

        # Default to hybrid (best balance)
        return "hybrid"

    def _standard_search(self, query, top_k):
        query_emb = self.embed_model.encode([query])
        scores = cosine_similarity(query_emb, self.embeddings)[0]
        top_idx = np.argsort(scores)[::-1][:top_k]

        return {
            "strategy": "standard",
            "results": [{"chunk": self.chunks[i], "metadata": self.metadata[i],
                        "score": float(scores[i])} for i in top_idx]
        }

    def _hybrid_search(self, query, top_k):
        # BM25
        bm25_scores = self.bm25.get_scores(query.lower().split())
        bm25_norm = bm25_scores / (max(bm25_scores) + 1e-6)

        # Vector
        query_emb = self.embed_model.encode([query])
        vector_scores = cosine_similarity(query_emb, self.embeddings)[0]

        # Combine
        combined = 0.35 * bm25_norm + 0.65 * vector_scores
        top_idx = np.argsort(combined)[::-1][:top_k * 3]

        # Re-rank
        candidates = [{"chunk": self.chunks[i], "metadata": self.metadata[i],
                      "score": float(combined[i])} for i in top_idx]

        pairs = [[query, c["chunk"]] for c in candidates]
        rerank_scores = self.reranker.predict(pairs)

        for i, c in enumerate(candidates):
            c["rerank_score"] = float(rerank_scores[i])

        results = sorted(candidates, key=lambda x: x["rerank_score"], reverse=True)

        return {
            "strategy": "hybrid",
            "results": results[:top_k]
        }

    def _hyde_search(self, query, top_k):
        # Generate hypothetical
        hypothetical = generate_hypothetical_answer(query)

        # Embed hypothetical
        hyde_emb = self.embed_model.encode([hypothetical])
        scores = cosine_similarity(hyde_emb, self.embeddings)[0]
        top_idx = np.argsort(scores)[::-1][:top_k]

        return {
            "strategy": "hyde",
            "hypothetical": hypothetical,
            "results": [{"chunk": self.chunks[i], "metadata": self.metadata[i],
                        "score": float(scores[i])} for i in top_idx]
        }

    def _self_rag_search(self, query, top_k):
        # Get initial results
        initial = self._hybrid_search(query, top_k * 2)

        # Evaluate relevance
        evaluated = []
        for r in initial["results"]:
            is_rel, reason = evaluate_relevance(query, r["chunk"])
            if is_rel:
                r["relevance_check"] = "passed"
                evaluated.append(r)

        return {
            "strategy": "self_rag",
            "evaluated_count": len(initial["results"]),
            "relevant_count": len(evaluated),
            "results": evaluated[:top_k]
        }

    def _crag_search(self, query, top_k):
        # Get initial results
        initial = self._hybrid_search(query, top_k)

        # Evaluate quality
        quality, reason = evaluate_retrieval_quality(query, initial["results"])

        result = {
            "strategy": "crag",
            "quality": quality,
            "evaluation": reason[:200] if isinstance(reason, str) else str(reason)[:200]
        }

        if quality == "correct":
            result["results"] = initial["results"]
        elif quality == "ambiguous":
            refined = refine_query(query)
            refined_results = self._hybrid_search(refined, top_k)
            result["refined_query"] = refined
            result["results"] = refined_results["results"]
        else:
            result["results"] = mock_web_search(query)
            result["fallback"] = "web_search"

        return result


# Test the unified class
print("\n" + "="*60)
print("🧪 Testing Unified AdvancedRAG Class")
print("="*60)

rag = AdvancedRAG(all_chunks, chunk_metadata)

test_cases = [
    ("What is Kevin Patel's role?", "auto"),
    ("How should I prepare for my performance review?", "auto"),
    ("What are the GDPR compliance requirements?", "auto"),
]

for query, strategy in test_cases:
    result = rag.search(query, strategy=strategy)
    print(f"\nQuery: {query}")
    print(f"   Strategy used: {result['strategy']}")
    print(f"   Results: {len(result['results'])}")

# ============================================================================
# 📝 STEP 10: SAVE PRODUCTION MODULE
# ============================================================================

## 📝 Step 10: Export Production Module

In [ ]:
production_code = '''
"""
Advanced RAG Module - Making AI Simple Day 14
Multiple retrieval strategies: Hybrid, HyDE, Self-RAG, CRAG

Usage:
    from advanced_rag import AdvancedRAG

    rag = AdvancedRAG(chunks, metadata)
    results = rag.search("your query", strategy="auto")

Strategies:
    - standard: Basic vector search
    - hybrid: BM25 + vector + re-ranking
    - hyde: Hypothetical document embeddings
    - self_rag: Self-reflective with evaluation
    - crag: Corrective with fallback
    - auto: Automatically choose best strategy
"""

from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class AdvancedRAG:
    def __init__(self, chunks, metadata, llm=None):
        self.chunks = chunks
        self.metadata = metadata
        self.llm = llm  # Optional: pass your LLM for HyDE/Self-RAG/CRAG

        self.embed_model = SentenceTransformer("all-MiniLM-L6-v2")
        self.reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
        self.embeddings = self.embed_model.encode(chunks)

        tokenized = [c.lower().split() for c in chunks]
        self.bm25 = BM25Okapi(tokenized)

    def search(self, query, strategy="hybrid", top_k=5):
        if strategy == "standard":
            return self._vector_search(query, top_k)
        elif strategy == "hybrid":
            return self._hybrid_search(query, top_k)
        elif strategy == "hyde" and self.llm:
            return self._hyde_search(query, top_k)
        else:
            return self._hybrid_search(query, top_k)

    def _vector_search(self, query, top_k):
        query_emb = self.embed_model.encode([query])
        scores = cosine_similarity(query_emb, self.embeddings)[0]
        top_idx = np.argsort(scores)[::-1][:top_k]
        return [{"chunk": self.chunks[i], "metadata": self.metadata[i],
                "score": float(scores[i])} for i in top_idx]

    def _hybrid_search(self, query, top_k):
        bm25_scores = self.bm25.get_scores(query.lower().split())
        bm25_norm = bm25_scores / (max(bm25_scores) + 1e-6)

        query_emb = self.embed_model.encode([query])
        vector_scores = cosine_similarity(query_emb, self.embeddings)[0]

        combined = 0.35 * bm25_norm + 0.65 * vector_scores
        top_idx = np.argsort(combined)[::-1][:top_k * 3]

        candidates = [{"chunk": self.chunks[i], "metadata": self.metadata[i],
                      "score": float(combined[i])} for i in top_idx]

        pairs = [[query, c["chunk"]] for c in candidates]
        rerank_scores = self.reranker.predict(pairs)

        for i, c in enumerate(candidates):
            c["rerank_score"] = float(rerank_scores[i])

        return sorted(candidates, key=lambda x: x["rerank_score"], reverse=True)[:top_k]

    def _hyde_search(self, query, top_k):
        hypothetical = self.llm.generate(
            f"Write a paragraph that would answer: {query}"
        )
        hyde_emb = self.embed_model.encode([hypothetical])
        scores = cosine_similarity(hyde_emb, self.embeddings)[0]
        top_idx = np.argsort(scores)[::-1][:top_k]
        return [{"chunk": self.chunks[i], "metadata": self.metadata[i],
                "score": float(scores[i]), "hypothetical": hypothetical}
                for i in top_idx]

if __name__ == "__main__":
    test_chunks = ["PTO is 15 days for new employees", "Security requires MFA"]
    test_meta = [{"id": i} for i in range(len(test_chunks))]
    rag = AdvancedRAG(test_chunks, test_meta)
    print(rag.search("vacation policy"))
'''

with open("advanced_rag.py", "w") as f:
    f.write(production_code)

print("   Saved: advanced_rag.py")
print("   Download and use in your projects!")

